# Import Libray

In [3]:
import cv2
import numpy as np
from PIL import Image
import os
import pathlib

# Organize Data

In [25]:
current_path = pathlib.Path('.')

In [87]:

def getFaceSampelsAndIds(imagePaths,face_label):
    faceSamples = []
    ids = []  
    for imagePath in imagePaths:
        ext  = os.path.split(imagePath)[-1].split(".")[1]
        if ext !='DS_Store': #ignore .DS_Store in MacOS folder
            PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
            img_numpy = np.array(PIL_img,'uint8')   
            faces = detector.detectMultiScale(img_numpy)

            for (x,y,w,h) in faces:
                faceSamples.append(img_numpy[y:y+h,x:x+w])
                ids.append(face_label)
    return ids, faceSamples
        

def getImagesAndLabels():
    faceSamples=[]
    ids = []
    i=0
    for item in current_path.glob('Data/*'):
        if str(item).split('/')[-1] != '.DS_Store': #ignore .DS_Store in MacOS folder
            i += 1
            temp_ids = []
            temp_faceSamples = []
            imagePaths = [os.path.join(item,f) for f in os.listdir(str(item))]
            temp_ids,temp_faceSamples = getFaceSampelsAndIds(imagePaths,i)
            faceSamples.extend(temp_faceSamples)
            ids.extend(temp_ids)
    return ids,faceSamples

# TRAIN MODEL 

In [ ]:
# Define model
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");

In [88]:
# Run Train
print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
ids, faces = getImagesAndLabels()
recognizer.train(faces, np.array(ids))


 [INFO] Training faces. It will take a few seconds. Wait ...


In [89]:
# Save the model into trainer/trainer.yml
recognizer.write('trainer/trainer.yml') # recognizer.save() worked on Mac, but not on Pi

# Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))


 [INFO] 3 faces trained. Exiting Program


# Real-time face recognition

In [90]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

font = cv2.FONT_HERSHEY_SIMPLEX

#iniciate id counter
id = 0

# names related to ids: example ==> Marcelo: id=1,  etc
names = ['None', 'Tam', 'Nguyen', 'Thu'] 

In [92]:
# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height

# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)
prev_id=''
while True:

    ret, img =cam.read()
    #img = cv2.flip(img, -1) # Flip vertically

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale( 
        img,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH)),
       )
    
    for(x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w,y+h), (221,228,232), 1)
        prev_id = id
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

        # Check if confidence is less them 60 ==> "0" is perfect match 
        if (confidence < 60):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        cv2.putText(img, str(id), (x+5,y-5), font, 1, (221,228,232), 2)
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
        
    cv2.imshow('camera',img) 

    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break

# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()


 [INFO] Exiting Program and cleanup stuff
